# Item-based Models for Movielens

In this file we are going to proceed with the building in the models. Machine learning models to build recommender systems. The models that are going to be built are Collaborative filtering using Item-based rating prediction (ItemKNN) and Item-based classification (ItemKNN).

Collaborative filtering is a technique used in recommendation systems to predict or classify items based on the preferences or behavior of similar users or items. Item-based Collaborative Filtering (CF) focuses on the similarity between items rather than users. There are two main approaches within item-based CF: rating prediction and classification.

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # to make jupyter print all outputs, not just the last one
from IPython.core.display import HTML # to pretty print pandas df and be able to copy them over (e.g. to ppt slides)

We import the necessary libraries for the modelling:

- Pandas: Data manipulation and analysis library in Python, primarily for structured data using Series and DataFrame.

- NumPy: Fundamental library for numerical computing with support for large arrays and mathematical functions.

- os: Python module for interacting with the operating system, used for file and directory operations.

In [2]:
# We import the necessary libraries
import pandas as pd
import numpy as np
import os

In [3]:
# We print the directory where the file is located
print(os.getcwd())

c:\Users\Jaume\Documents\MDDB\SDM\SDfM---Jaume-and-Stijn 2\SDfM---Jaume-and-Stijn


In [4]:
# We set the directory to the cleaned folder
os.listdir(os.path.join('.', 'cleaned'))

['final_sample30_parquet',
 'final_sample50_parquet',
 'final_sample5_parquet',
 'final_strat_sample_parquet',
 'movielens_parquet',
 'movielens_parquet_small',
 'netflix_parquet',
 'sample_tenth_netflix',
 'strat_sample_movielens',
 'strat_sample_netflix']

In [5]:
# We read the final_sample file and store it in a dataframe
df = pd.read_parquet('cleaned/final_strat_sample_parquet')

In [6]:
# We print shape of the dataframe
df.shape

(50381, 25)

In [7]:
# We print the first 5 rows of the dataframe
df.head()

,movieId,title,year,userId,rating,date,Drama,Thriller,Crime,Mystery,...,Musical,Romance,Sci-Fi,Children,Action,Horror,War,IMAX,Western,(no genres listed)
0,143367,Silence,2016,267711,3.5,2022-05-01,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,143367,Silence,2016,9138,4.5,2021-08-18,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,143367,Silence,2016,9138,4.5,2021-08-18,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,143367,Silence,2016,306765,3.5,2017-11-06,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,143367,Silence,2016,318227,2.5,2019-05-03,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# We print the columns of the dataframe
df.columns

Index(['movieId', 'title', 'year', 'userId', 'rating', 'date', 'Drama',
       'Thriller', 'Crime', 'Mystery', 'Documentary', 'Adventure', 'Animation',
       'Comedy', 'Fantasy', 'Musical', 'Romance', 'Sci-Fi', 'Children',
       'Action', 'Horror', 'War', 'IMAX', 'Western', '(no genres listed)'],
      dtype='object')

In [9]:
# We print the number of unique users and movies
print(df['userId'].nunique())
print(df['movieId'].nunique())


37841
500


In [10]:
# Check if there are userIDs that haven't rated movies at all
print(df['userId'].value_counts().sort_values(ascending=False))

# Check if there are movieIDs that haven't been rated at all
print(df['movieId'].value_counts().sort_values(ascending=False))

userId
215490    378
137062     43
9138       33
189614     31
222223     26
         ... 
141435      1
215467      1
114372      1
245678      1
158343      1
Name: count, Length: 37841, dtype: int64
movieId
1221      5328
733       4590
48516     4190
208       3747
89745     3326
          ... 
92760        1
119165       1
184849       1
229975       1
103570       1
Name: count, Length: 500, dtype: int64


In [11]:
# Check if there are NaN values in the rating column
df['rating'].isna().sum()

0

We check the number of different movies that we have in the dataset and the number of unique users we have.

In [12]:
# Print the number of unique users
print("Number of unique users:", df['userId'].nunique())

# Print the number of unique movies
print("Number of unique movies:", df['movieId'].nunique())


Number of unique users: 37841
Number of unique movies: 500


We see that we have `25123` unique users and `400` movies, a significant number to train the model and recommend apropiate movies.

In [13]:
# print the data types of the DF
df.dtypes

movieId                 int64
title                  object
year                   object
userId                  int64
rating                float64
date                   object
Drama                   int64
Thriller                int64
Crime                   int64
Mystery                 int64
Documentary             int64
Adventure               int64
Animation               int64
Comedy                  int64
Fantasy                 int64
Musical                 int64
Romance                 int64
Sci-Fi                  int64
Children                int64
Action                  int64
Horror                  int64
War                     int64
IMAX                    int64
Western                 int64
(no genres listed)      int64
dtype: object

We have to distinguish the columns that are part of the genres from the ones that aren't since we are going to also use the genres in this model to calculate the similarity between items.

In [14]:
# We select the columns that are not genres
columns_not_genres = ['movieId', 'title', 'year', 'userId', 'rating', 'date', '(no genres listed)']
for column in df.columns:
    if column not in columns_not_genres :
        df.rename(columns={column: 'genre_' + column}, inplace=True)


In [15]:
# We print the columns of the dataframe
df.columns

Index(['movieId', 'title', 'year', 'userId', 'rating', 'date', 'genre_Drama',
       'genre_Thriller', 'genre_Crime', 'genre_Mystery', 'genre_Documentary',
       'genre_Adventure', 'genre_Animation', 'genre_Comedy', 'genre_Fantasy',
       'genre_Musical', 'genre_Romance', 'genre_Sci-Fi', 'genre_Children',
       'genre_Action', 'genre_Horror', 'genre_War', 'genre_IMAX',
       'genre_Western', '(no genres listed)'],
      dtype='object')

Now we get rid of the cold start problem, we are not gonna give recommendation to the users if they haven't reviewed at least 3 movies.

In [16]:
# Group the DataFrame by 'userId' and count the number of ratings per user
user_rating_counts = df.groupby('userId').size()

# Filter out users who have rated less than 3 movies
active_users = user_rating_counts[user_rating_counts >= 3].index.tolist()

# Select rows from the original DataFrame for active users
df = df[df['userId'].isin(active_users)]

# Display the filtered DataFrame
df.head()a

,movieId,title,year,userId,rating,date,genre_Drama,genre_Thriller,genre_Crime,genre_Mystery,...,genre_Musical,genre_Romance,genre_Sci-Fi,genre_Children,genre_Action,genre_Horror,genre_War,genre_IMAX,genre_Western,(no genres listed)
0,143367,Silence,2016,267711,3.5,2022-05-01,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,143367,Silence,2016,9138,4.5,2021-08-18,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,143367,Silence,2016,9138,4.5,2021-08-18,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,143367,Silence,2016,306765,3.5,2017-11-06,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,143367,Silence,2016,58551,3.0,2017-01-27,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


# Item-based Rating Prediction (ItemKNN)

### Step 1.1: User-Item matrix construction
The first thing we have to do is build the user-item matrix:

- Choose a similarity metric to calculate the similarity between items. Common metrics include cosine similarity, Pearson correlation coefficient, and Jaccard similarity. We are going to use the first one of them, the cosine similarity.

##### Libraries Used
- **pandas**: A powerful data manipulation library in Python.
- **numpy**: A library for numerical computing in Python.

In [17]:
import pandas as pd
import numpy as np

# Convert userId column to integer type if needed
if df['userId'].dtype != int:
    df['userId'] = df['userId'].astype(int)

# Get all unique user IDs and movie IDs
all_user_ids = np.unique(df['userId'])
all_movie_ids = np.unique(df['movieId'])

# Determine the number of unique users and movies
num_users = len(all_user_ids)
num_movies = len(all_movie_ids)

# Create a dictionary to store ratings for each user
user_ratings = {}

# Iterate through the DataFrame and populate the user_ratings dictionary
for _, row in df.iterrows():
    user_id = row['userId']
    movie_id = row['movieId']
    rating = row['rating']
    
    # Initialize the user's ratings list if not already present
    if user_id not in user_ratings:
        user_ratings[user_id] = np.zeros(num_movies)
    
    # Assign the rating to the corresponding movie index
    user_ratings[user_id][np.where(all_movie_ids == movie_id)[0][0]] = rating

# Create user-item matrix from the dictionary
item_user_matrix = pd.DataFrame(user_ratings.values(), index=user_ratings.keys(), columns=all_movie_ids)

# Convert user-item matrix to NumPy array for faster computation
item_user_array = item_user_matrix.to_numpy()


In [18]:
item_user_matrix.head()

,75,77,135,208,290,413,461,733,792,810,...,271162,272579,274245,274495,277820,280482,283259,284167,284513,286031
267711,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
306765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58551,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
223459,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
item_user_array

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Train-test split

The train-val-test split is a technique used in machine learning to evaluate the performance of a model. It involves dividing the dataset into three subsets: the training set, the validation set, and the test set.

In this model we are going to devide the user-item interaction matrix into three since in this one, we already have all the items and we can already calculate the similarity for all the items with the train set.

The training set is used to train the model and optimize its parameters.
The validation set is used to fine-tune the model and select the best hyperparameters. The test set is used to evaluate the final performance of the model on unseen data.

By using a train-val-test split, we can assess the model's performance on unseen data and ensure that it generalizes well to new examples. It helps prevent overfitting and provides a more reliable estimate of the model's performance in real-world scenarios.

For the split we will use the `sklearn` library, more specifacally the `train_test_split` function. This function is commonly used in machine learning workflows to split a dataset into two subsets: one for training the model and another for testing the model's performance.

In [20]:
from sklearn.model_selection import train_test_split
# Split the data into training and test sets
train_val, test = train_test_split(item_user_array, test_size=0.2, random_state=42)

# Split the training set into training and validation sets
train, val = train_test_split(train_val, test_size=0.2, random_state=42)

# Print the shapes of the datasets
print("Training set shape:", train.shape)
print("Validation set shape:", val.shape)
print("Test set shape:", test.shape)

# We are also going to do the split for the matrix df
# Split the user-item matrix into training and test sets
train_val_matrix, test_matrix = train_test_split(item_user_matrix, test_size=0.2, random_state=42)

# Split the training set matrix into training and validation sets
train_matrix, val_matrix = train_test_split(train_val_matrix, test_size=0.2, random_state=42)

# We print a ' ' to give some space inbetween lines
print(' ')

# Print the shapes of the matrix datasets
print("Training set matrix shape:", train_matrix.shape)
print("Validation set matrix shape:", val_matrix.shape)
print("Test set matrix shape:", test_matrix.shape)


Training set shape: (1516, 435)
Validation set shape: (379, 435)
Test set shape: (474, 435)
 
Training set matrix shape: (1516, 435)
Validation set matrix shape: (379, 435)
Test set matrix shape: (474, 435)


We define mapping dictionaries that will be used to locate where the movies and users are in the different matrices created (train, df, test, item similarity...)

In [21]:

# Adjusting the mapping to start indexing from 0
user_id_to_index = {user_id: i for i, user_id in enumerate(item_user_matrix.index)}
index_to_user_id = {i: user_id for i, user_id in enumerate(item_user_matrix.index)}

# Create a mapping from movie IDs to indices
movie_id_to_index = {movie_id: i for i, movie_id in enumerate(item_user_matrix.columns)}
index_to_movie_id = {i: movie_id for i, movie_id in enumerate(item_user_matrix.columns)}



In [22]:
train_matrix

,75,77,135,208,290,413,461,733,792,810,...,271162,272579,274245,274495,277820,280482,283259,284167,284513,286031
120864,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91440,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58822,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19119,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
174808,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
210321,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
222135,0.0,0.0,0.0,2.5,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Step 1.2: Item-Genre matrix

In the following code snippet we will construct the item-genre matrix, that will consist of all the items in the dataset (we will input the ones in train to not use the test set, since we have all of them in train) and the genres of each of the items, with this matrix we will be able to compute using the cosine similarity the item-item similarity matrix based on the genres.

In [24]:
import numpy as np

# We create item-genre matrix
def create_item_genre_matrix(df, train):
    """
    Create item-genre matrix based on genre information in the dataframe.

    Parameters:
        df (pandas.DataFrame): DataFrame containing movie genre information.
        train (pandas.DataFrame): DataFrame containing user-item ratings in the train set.

    Returns:
        numpy.ndarray: Item-genre matrix.
    """
    # Extract genre columns from the dataframe
    genre_columns = [col for col in df.columns if col.startswith('genre_')]
    
    # Extract item IDs
    item_ids = train.columns
    
    # Initialize item-genre matrix with zeros
    item_genre_matrix = np.zeros((len(genre_columns), len(item_ids)))
    
    # Fill the matrix with genre information
    for i, row in df.iterrows():
        item_id = row['movieId']  # Assuming 'movieId' is the column containing item IDs
        if item_id in item_ids:
            item_index = np.where(item_ids == item_id)[0][0]
            genres = row[genre_columns].values
            item_genre_matrix[:, item_index] = genres
    
    return item_genre_matrix

# We create the item_genre matrix with the train set and the original df
item_genre_matrix = create_item_genre_matrix(df, train_matrix)


In [25]:
item_genre_matrix

item_genre_matrix.shape

array([[1., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

(18, 435)

### Step 2: Calculate Similarities

In this step of the model we are going to define the `cosine_similarity` function, that will take in two vectors and will calculate the similarity between them.

In [26]:
# Calculate cosine similarity between items using NumPy functions
def cosine_similarity(a, b):
    """
    Calculate cosine similarity between two vectors.

    Parameters:
        a (numpy.ndarray): First vector.
        b (numpy.ndarray): Second vector.

    Returns:
        float: Cosine similarity between the two vectors.
    """
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    similarity = dot_product / (norm_a * norm_b)
    return similarity


#### Collaborative Filtering with Cosine Similarity

This code snippet demonstrates how to perform item-based collaborative filtering using cosine similarity for the ratings interactions. We are going to predict a user's preferences for items based on the preferences of similar users/items.


In [27]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def calculate_item_similarity_matrix(data):
    """
    Calculate item-item similarity matrix based on user-item ratings using cosine similarity.

    Parameters:
        data (numpy.ndarray or scipy.sparse matrix): User-item ratings matrix.

    Returns:
        numpy.ndarray: Item-item similarity matrix.
    """
    # Calculate cosine similarity between items
    item_similarity_matrix = cosine_similarity(data.T)
    np.fill_diagonal(item_similarity_matrix, 1)  # Set diagonal values to 1
    return item_similarity_matrix

# We calculate the item similarity using the train user-item matrix data
item_similarity_matrix_train = calculate_item_similarity_matrix(train)
print(item_similarity_matrix_train)


[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


Now we have to repeat the same process but for the item-genres matrix to have a matrix with the similarity between items.

In [28]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

def calculate_item_similarity_genres(item_genre_matrix):
    """
    Calculate item-item similarity based on genre information using cosine similarity.

    Parameters:
        item_genre_matrix (numpy.ndarray): Matrix containing item-genre information.

    Returns:
        numpy.ndarray: Matrix containing item-item similarity scores.
    """
    # Convert the item-genre matrix to a sparse representation
    sparse_item_genre_matrix = sparse.csr_matrix(item_genre_matrix)
    
    # Calculate cosine similarity between items
    item_similarity_matrix = cosine_similarity(sparse_item_genre_matrix.T)
    
    # Set diagonal values to 1
    np.fill_diagonal(item_similarity_matrix, 1)
    
    return item_similarity_matrix

# Example usage:
# Assuming item_genre_matrix is the item-genre matrix
item_similarity_genres_matrix = calculate_item_similarity_genres(item_genre_matrix)
print(item_similarity_genres_matrix)


[[1.         0.         0.70710678 ... 0.5        0.40824829 0.        ]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.70710678 0.         1.         ... 0.         0.         0.        ]
 ...
 [0.5        0.         0.         ... 1.         0.40824829 0.        ]
 [0.40824829 0.         0.         ... 0.40824829 1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [29]:
item_similarity_genres_matrix.shape

(435, 435)

In [30]:
movie_id_to_index

{75: 0,
 77: 1,
 135: 2,
 208: 3,
 290: 4,
 413: 5,
 461: 6,
 733: 7,
 792: 8,
 810: 9,
 857: 10,
 901: 11,
 955: 12,
 1184: 13,
 1221: 14,
 1352: 15,
 1365: 16,
 1507: 17,
 1656: 18,
 1662: 19,
 1701: 20,
 1713: 21,
 1719: 22,
 1878: 23,
 1901: 24,
 2297: 25,
 2340: 26,
 2535: 27,
 2662: 28,
 2750: 29,
 2857: 30,
 2871: 31,
 2986: 32,
 3081: 33,
 3218: 34,
 3252: 35,
 3452: 36,
 3468: 37,
 3495: 38,
 3746: 39,
 3789: 40,
 3933: 41,
 4051: 42,
 4267: 43,
 4347: 44,
 4390: 45,
 4402: 46,
 4442: 47,
 4528: 48,
 4541: 49,
 4695: 50,
 4880: 51,
 4915: 52,
 5176: 53,
 5201: 54,
 5303: 55,
 5511: 56,
 5536: 57,
 5864: 58,
 5974: 59,
 6140: 60,
 6174: 61,
 6236: 62,
 6375: 63,
 6449: 64,
 6793: 65,
 7120: 66,
 7190: 67,
 7390: 68,
 7523: 69,
 7786: 70,
 7802: 71,
 7813: 72,
 7837: 73,
 7921: 74,
 7995: 75,
 8259: 76,
 8375: 77,
 8400: 78,
 8427: 79,
 8447: 80,
 8664: 81,
 8683: 82,
 8728: 83,
 25767: 84,
 25822: 85,
 26148: 86,
 26707: 87,
 26803: 88,
 27192: 89,
 27334: 90,
 27392: 91,
 2755

In [31]:
print(index_to_user_id)

{0: 267711, 1: 9138, 2: 306765, 3: 58551, 4: 223459, 5: 151122, 6: 222223, 7: 92632, 8: 127917, 9: 215896, 10: 144502, 11: 52796, 12: 78255, 13: 68383, 14: 40576, 15: 48068, 16: 68955, 17: 197102, 18: 143805, 19: 134404, 20: 312342, 21: 91970, 22: 63344, 23: 240754, 24: 49000, 25: 137062, 26: 134034, 27: 167423, 28: 84092, 29: 205032, 30: 117959, 31: 175730, 32: 166377, 33: 178768, 34: 248221, 35: 103894, 36: 196960, 37: 41417, 38: 220599, 39: 316174, 40: 126357, 41: 319553, 42: 272129, 43: 270802, 44: 155789, 45: 198544, 46: 160144, 47: 70160, 48: 330196, 49: 128787, 50: 165356, 51: 30839, 52: 202311, 53: 262043, 54: 321624, 55: 187748, 56: 170347, 57: 112343, 58: 177726, 59: 262371, 60: 105053, 61: 91440, 62: 144749, 63: 307321, 64: 50012, 65: 176121, 66: 274202, 67: 190682, 68: 289935, 69: 272358, 70: 105804, 71: 171442, 72: 183281, 73: 76618, 74: 274406, 75: 178717, 76: 157479, 77: 130007, 78: 116702, 79: 68146, 80: 315511, 81: 61822, 82: 230765, 83: 232645, 84: 104198, 85: 314043,

In [32]:
print(user_id_to_index)

{267711: 0, 9138: 1, 306765: 2, 58551: 3, 223459: 4, 151122: 5, 222223: 6, 92632: 7, 127917: 8, 215896: 9, 144502: 10, 52796: 11, 78255: 12, 68383: 13, 40576: 14, 48068: 15, 68955: 16, 197102: 17, 143805: 18, 134404: 19, 312342: 20, 91970: 21, 63344: 22, 240754: 23, 49000: 24, 137062: 25, 134034: 26, 167423: 27, 84092: 28, 205032: 29, 117959: 30, 175730: 31, 166377: 32, 178768: 33, 248221: 34, 103894: 35, 196960: 36, 41417: 37, 220599: 38, 316174: 39, 126357: 40, 319553: 41, 272129: 42, 270802: 43, 155789: 44, 198544: 45, 160144: 46, 70160: 47, 330196: 48, 128787: 49, 165356: 50, 30839: 51, 202311: 52, 262043: 53, 321624: 54, 187748: 55, 170347: 56, 112343: 57, 177726: 58, 262371: 59, 105053: 60, 91440: 61, 144749: 62, 307321: 63, 50012: 64, 176121: 65, 274202: 66, 190682: 67, 289935: 68, 272358: 69, 105804: 70, 171442: 71, 183281: 72, 76618: 73, 274406: 74, 178717: 75, 157479: 76, 130007: 77, 116702: 78, 68146: 79, 315511: 80, 61822: 81, 230765: 82, 232645: 83, 104198: 84, 314043: 85,

In [33]:
item_user_array.shape

(2369, 435)

In [34]:
item_user_matrix.shape

(2369, 435)

In [35]:
item_similarity_matrix_train.shape

(435, 435)

### Matrices Summation:

Once we have both similarity matrices (the one from the ratings and the one from the genres) we can add them with a weighted sumation.

In [36]:
import numpy as np

def combine_similarity_matrices(similarity_matrix1, similarity_matrix2, weight1=0.7, weight2=0.3):
    """
    Combine two similarity matrices using weighted summation.

    Parameters:
        similarity_matrix1 (numpy.ndarray): First similarity matrix.
        similarity_matrix2 (numpy.ndarray): Second similarity matrix.
        weight1 (float): Weight for the first similarity matrix (default: 0.7).
        weight2 (float): Weight for the second similarity matrix (default: 0.3).

    Returns:
        numpy.ndarray: Combined similarity matrix.
    """
    # Check if the matrices have the same shape
    if similarity_matrix1.shape != similarity_matrix2.shape:
        raise ValueError("Matrices must have the same shape.")
    
    # Combine matrices using weighted summation directly without creating an intermediate array
    np.multiply(similarity_matrix1, weight1, out=similarity_matrix1)
    np.multiply(similarity_matrix2, weight2, out=similarity_matrix2)
    np.add(similarity_matrix1, similarity_matrix2, out=similarity_matrix1)
    return similarity_matrix1

# Define weights for each matrix
weight_ratings = 0.7
weight_genres = 0.3

# Combine the similarity matrices
item_similarity_matrix_train = combine_similarity_matrices(item_similarity_matrix_train, item_similarity_genres_matrix, weight_ratings, weight_genres)

# Print or use the combined similarity matrix as needed
print("Combined Similarity Matrix:\n", item_similarity_matrix_train)


Combined Similarity Matrix:
 [[1.         0.         0.21213203 ... 0.15       0.12247449 0.        ]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.21213203 0.         1.         ... 0.         0.         0.        ]
 ...
 [0.15       0.         0.         ... 1.         0.12247449 0.        ]
 [0.12247449 0.         0.         ... 0.12247449 1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


### Step 3: Neighborhood Selection
- Determine the neighborhood size, i.e., the number of most similar items to consider when predicting ratings for a target item.
- Select the most similar items for each item in the dataset based on their calculated similarities. This forms the neighborhood for each item.

#### Item-Based Neighborhoods and Ratings Aggregation

This code following snippet enhances the previous item-based collaborative filtering approach by considering ratings aggregation within the item neighborhoods.

##### Steps:

1. **Defining Neighborhood Size**:
   - The variable `neighborhood_size` determines the number of most similar items to consider in the neighborhood.

2. **Initializing Data Structure**:
   - An empty dictionary `item_neighborhoods` is initialized to store the neighborhoods for each item.

3. **Iterating Over Items**:
   - For each movie in the dataset:
     - All ratings for the current movie are extracted from the DataFrame (`df`).
     - Ratings aggregation is performed. In this example, the average rating for the movie is computed, but other aggregation methods can be used.
     - The similarity scores for the current movie are retrieved from the precomputed `item_similarity_matrix`.
     - Similarity scores are sorted in descending order, and the indices of the most similar items (excluding itself) are obtained.
     - These indices are converted back to movie IDs, forming the neighborhood for the current item.
     - The neighborhood for the current item is stored in the `item_neighborhoods` dictionary.

4. **Output**:
   - `item_neighborhoods`: A dictionary where keys are movie IDs, and values are lists of movie IDs representing the neighborhood of each item. Each movie's neighborhood includes movies with similar ratings and content.

##### Note:
- This approach considers both similarity in ratings and content (as captured by cosine similarity) when building item neighborhoods.
- Aggregating ratings within item neighborhoods helps in providing more personalized recommendations.


In [37]:
import numpy as np

def neighborhood_selection(item_user_array, index_to_movie_id, item_similarity_matrix, neighborhood_size=5):
    """
    Selects item neighborhoods based on similarity scores.

    Parameters:
        item_user_array (numpy.ndarray): Matrix containing user-item ratings.
        index_to_movie_id (dict): Dictionary mapping item indices to movie IDs.
        item_similarity_matrix (numpy.ndarray): Matrix containing item-item similarity scores.
        neighborhood_size (int): Number of most similar items to include in the neighborhood.

    Returns:
        dict: Dictionary containing item neighborhoods for each movie.
    """
    
    # Initialize an empty dictionary to store item neighborhoods
    item_neighborhoods = {}

    # Iterate over each item (movie) index in the dataset
    for movie_index in range(item_user_array.shape[1]):
        # Convert the item index to movie ID
        movie_id = index_to_movie_id[movie_index]

        # Extract all ratings for the current movie
        movie_ratings = item_user_array[:, movie_index]

        # Aggregate ratings (e.g., compute the mean rating)
        movie_avg_rating = np.mean(movie_ratings)

        # Retrieve similarity scores for the current movie
        similarity_scores = item_similarity_matrix[movie_index]

        # Sort similarity scores in descending order and get indices of most similar items
        most_similar_indices = np.argsort(similarity_scores)[::-1][1:neighborhood_size+1]

        # Convert indices back to movie IDs to form the neighborhood
        neighborhood = [index_to_movie_id[idx] for idx in most_similar_indices]

        # Store the neighborhood for the current item in the item_neighborhoods dictionary
        item_neighborhoods[movie_id] = neighborhood

    return item_neighborhoods

# Example usage:
item_neighborhoods_train = neighborhood_selection(item_user_array, index_to_movie_id, item_similarity_matrix_train)
print(item_neighborhoods_train)


{75: [6793, 189011, 215759, 3495, 207534], 77: [48741, 1901, 184133, 220056, 91762], 135: [149352, 413, 47640, 4402, 165119], 208: [89745, 87520, 73321, 733, 61248], 290: [1221, 39292, 3746, 1719, 192875], 413: [149352, 172311, 47640, 135, 165119], 461: [1656, 248038, 112944, 31903, 7837], 733: [61248, 286031, 2871, 7802, 68319], 792: [4051, 810, 4915, 8447, 147694], 810: [792, 3495, 1713, 4051, 163014], 857: [1184, 47099, 183087, 205913, 71638], 901: [71416, 33312, 107682, 58740, 274245], 955: [1878, 91628, 5176, 108078, 162488], 1184: [3495, 857, 1701, 2750, 3789], 1221: [48516, 39292, 290, 284167, 61323], 1352: [195247, 6174, 1662, 132144, 7523], 1365: [284167, 3789, 8664, 3468, 3252], 1507: [27771, 45072, 26707, 226308, 251900], 1656: [27857, 461, 267434, 92874, 170311], 1662: [184427, 1352, 136936, 1221, 214138], 1701: [4347, 1184, 167732, 2750, 61323], 1713: [163014, 83049, 135, 810, 6793], 1719: [177469, 4880, 3468, 155074, 102354], 1878: [181211, 955, 1184, 78270, 91628], 1901:

### Step 4: Rating Prediction

- For each target item and user pair where the user hasn't rated the target item:
Identify the neighborhood of similar items to the target item.
- Predict the rating for the target item using a weighted average of the ratings of the items in its neighborhood, where the weights are the similarities between the items and the target item.
- Adjust the prediction based on the user's average rating or other normalization techniques, if necessary.

In [38]:
# Initialize an empty DataFrame to store predicted ratings
predicted_ratings_train = pd.DataFrame(index=train_matrix.index, columns=train_matrix.columns)

# Iterate over each user-item pair in the training set
for user_id, user_ratings in train_matrix.iterrows():
    for movie_id, rating in user_ratings.items():
        
        # Check if the movie has a neighborhood defined
        if movie_id in item_neighborhoods_train:
            neighborhood = item_neighborhoods_train[movie_id]
            
            # Filter out movies from the neighborhood that the user has rated in the training set
            filtered_neighborhood = [neighbor_movie_id for neighbor_movie_id in neighborhood if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0]
            
            # Check if there are valid indices in the filtered neighborhood
            if len(filtered_neighborhood) > 0:
                # Calculate the predicted rating for the target movie based on the neighborhood
                neighbor_ratings = [user_ratings.loc[neighbor_movie_id] for neighbor_movie_id in filtered_neighborhood]
                
                # Calculate the mean rating using only the movies in the neighborhood that have been rated by the user
                predicted_rating = np.mean(neighbor_ratings)
            else:
                # If the filtered neighborhood is empty, assign the mean rating of all movies rated by the user
                predicted_rating = user_ratings[user_ratings != 0].mean()
            
            # Assign the predicted rating to the corresponding cell in the DataFrame
            predicted_ratings_train.at[user_id, movie_id] = predicted_rating

# Fill NaN values with mean ratings across all users
predicted_ratings_train.fillna(predicted_ratings_train.mean().mean(), inplace=True)

# Display the head of the predicted_ratings DataFrame
predicted_ratings_train.head()


,75,77,135,208,290,413,461,733,792,810,...,271162,272579,274245,274495,277820,280482,283259,284167,284513,286031
120864,3.0,3.0,3.0,5.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,5.0,3.0,3.0,3.0
91440,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,...,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5
58822,3.1,3.1,3.1,3.0,3.5,3.1,3.1,4.0,3.1,3.1,...,2.0,4.5,3.1,3.1,3.1,3.1,3.0,3.5,3.1,3.1
19119,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,...,3.333333,3.333333,3.333333,3.0,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333
174808,2.5,2.75,3.166667,2.75,2.75,3.25,2.75,2.75,2.75,2.75,...,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75,2.75


In [39]:
# We count the NaN values in the predicted_ratings_train dataframe
predicted_ratings_train.isnull().sum().sum()

0

We are now going to define a function that gets the top recomendations per user using the predicted ratings.

In [40]:
def get_top_recommendations(user_id, predicted_ratings, df):
    """
    Get top movie recommendations for a given user using predicted ratings.

    Parameters:
        user_id (int): ID of the user for whom recommendations are to be generated.
        predicted_ratings (pd.DataFrame): DataFrame containing predicted ratings for users and movies.
        df (pd.DataFrame): DataFrame containing movie ratings.

    Returns:
        list: Top movie titles recommended for the user.
    """
    # Ensure that predicted_ratings is a DataFrame
    if not isinstance(predicted_ratings, pd.DataFrame):
        raise ValueError("predicted_ratings must be a pandas DataFrame")

    # Check if the user ID exists in the predicted ratings DataFrame's index
    if user_id not in predicted_ratings.index:
        # If the user ID doesn't exist, return an empty list
        return ["The user doesn't exist"]

    # Get the predicted ratings for the user
    user_predicted_ratings = predicted_ratings.loc[user_id]
    
    # Filter out the movies that the user has already seen
    seen_movies = set(df[df['userId'] == user_id]['movieId'])
    unseen_movies = [movie_id for movie_id in user_predicted_ratings.index if movie_id not in seen_movies]
    
    # Check if there are unseen movies with predicted ratings
    if not unseen_movies:
        # If all movies have been seen, return an empty list
        return []
    
    # Sort the unseen movies by predicted rating in descending order
    sorted_unseen_movies = user_predicted_ratings[unseen_movies].sort_values(ascending=False)
    
    # Get the top 5 movie titles
    top_movie_ids = sorted_unseen_movies.head(5).index.tolist()
    
    # Get the unique movie titles corresponding to the top movie IDs
    top_movie_titles = set(df[df['movieId'].isin(top_movie_ids)]['title'])
    
    return list(top_movie_titles)[:5]  # Return only the first 5 unique movie titles


And we do a loop through all the users in the train data to get their recommended movies.

In [41]:
top_recommendations = []

# Iterate over each row (user) in the train_matrix
for user_id in train_matrix.index:
    # Get the actual user ID corresponding to the user index
    recommendations = get_top_recommendations(user_id, predicted_ratings_train, df)
    top_recommendations.append(recommendations)

# Output: List of top movie recommendations for each user in the train matrix
top_recommendations

# print size of top_recommendations
len(top_recommendations)


[['Transformers: Dark of the Moon ',
  'Book of Eli, The ',
  'I Am So Proud of You ',
  'Awake ',
  'Legion of Super-Heroes '],
 ['Truth and Justice ',
  'La cripta ',
  'Intruder in the Dust ',
  'Nine Days ',
  'Hatful of Rain, A '],
 ['The Cleveland Kidnappings ',
  'Deconstructing Harry ',
  'The Jade Mask ',
  'Adrenaline ',
  "President's Last Bang, The (Geuddae geusaramdeul) "],
 ['Big Bully ',
  'Joe Versus the Volcano ',
  "New Year's Eve ",
  'Unconscious (Inconscientes) ',
  'Woo '],
 ['Village People ',
  'Balls Out ',
  'Over the Brooklyn Bridge ',
  'Siberia ',
  'Job, czyli ostatnia szara komórka '],
 ['Lokis. A Manuscript of Professor Wittembach ',
  'Tehran: City of Love ',
  'Sleepy Hollow ',
  'Matchmaking ',
  'Christmas at Dollywood '],
 ['Godzilla vs. Gigan (Chikyû kogeki meirei: Gojira tai Gaigan) ',
  'Thief of Bagdad, The ',
  'Heller in Pink Tights ',
  'Iron Man & Captain America: Heroes United ',
  'Swordsman II (Legend of the Swordsman, The) (Xiao ao jiang

1516

### Step 5: Model Evaluation

We evaluate the performance of the ItemKNN algorithm using appropriate evaluation metrics such as Root Mean Squared Error (RMSE) and Mean Absolute Error (MAE)

In [42]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

def evaluate_model(train_matrix, predicted_ratings):
    """
    Evaluate the model's performance on the training data.

    Parameters:
        train_matrix (numpy.ndarray): Item-user matrix from the training data.
        predicted_ratings (pandas.DataFrame or numpy.ndarray): Predicted ratings DataFrame or array for the training data.

    Returns:
        float: Mean Absolute Error (MAE) on the training data.
        float: Root Mean Squared Error (RMSE) on the training data.
    """
    # Convert predicted_ratings to a numpy array if it's a DataFrame
    if isinstance(predicted_ratings, pd.DataFrame):
        predicted_ratings = predicted_ratings.to_numpy()

    # Ensure train_matrix and predicted_ratings have the same shape
    assert train_matrix.shape == predicted_ratings.shape, "Shapes of train_matrix and predicted_ratings are not consistent."

    # Initialize lists to store true and predicted ratings
    true_ratings = []
    pred_ratings = []

    # Iterate over each user and their ratings
    for user_id, user_ratings in enumerate(train_matrix):
        for movie_id, rating in enumerate(user_ratings):
            # Skip unrated movies
            if rating == 0:
                continue

            # Check if the indices are within the bounds of the predicted ratings array
            if user_id < predicted_ratings.shape[0] and movie_id < predicted_ratings.shape[1]:
                # Get the predicted rating for the corresponding movie
                pred_rating = predicted_ratings[user_id, movie_id]

                # Append the true and predicted ratings
                true_ratings.append(rating)
                pred_ratings.append(pred_rating)

    # Convert lists to numpy arrays
    true_ratings = np.array(true_ratings)
    pred_ratings = np.array(pred_ratings)

    # Calculate evaluation metrics
    mae = mean_absolute_error(true_ratings, pred_ratings)
    rmse = np.sqrt(mean_squared_error(true_ratings, pred_ratings))

    return mae, rmse

# Assuming train is the numpy array or DataFrame representing the item-user matrix
# Assuming predicted_ratings_train is the DataFrame or numpy array representing the predicted ratings for the training data

# Evaluate the model
mae, rmse = evaluate_model(train, predicted_ratings_train)

print("Mean Absolute Error (MAE) on Training Data:", mae)
print("Root Mean Squared Error (RMSE) on Training Data:", rmse)


Mean Absolute Error (MAE) on Training Data: 0.6843011908435963
Root Mean Squared Error (RMSE) on Training Data: 0.9113101029110728


### Step 6: Parameter Tuning
- Experiment with different parameters such as similarity threshold, neighborhood size, and similarity metric to optimize the performance of the ItemKNN algorithm.
- Use cross-validation or other techniques to tune these parameters and avoid overfitting.

1. **Import Necessary Libraries:**
   - We import the required libraries for performing grid search cross-validation (`GridSearchCV`), creating custom scorers (`make_scorer`), and utilizing the `NearestNeighbors` algorithm.

2. **Define Cosine Similarity Function:**
   - We define a custom function `cosine_similarity` to compute the cosine similarity between two vectors. This function calculates the dot product of the vectors and divides it by the product of their norms.

3. **Define Custom Scorer:**
   - We create a custom scorer `cosine_similarity_scorer` using `make_scorer`, which enables us to use cosine similarity as the scoring metric during grid search cross-validation.

4. **Define Parameter Grid:**
   - We specify a parameter grid `param_grid` containing the hyperparameters to be tuned. In this case, we're tuning the number of neighbors (`n_neighbors`) and the distance metric (`metric`) for the `NearestNeighbors` algorithm.

5. **Initialize NearestNeighbors Model:**
   - We initialize the `NearestNeighbors` model without specifying any hyperparameters.

6. **Create GridSearchCV Object:**
   - We create a `GridSearchCV` object named `grid_search` with the specified parameter grid, cross-validation strategy (5-fold cross-validation), and custom scoring metric (`cosine_similarity_scorer`).

7. **Fit the Data:**
   - We fit the `item_user_matrix` data to the `grid_search` object to perform hyperparameter tuning. `item_user_matrix` typically contains the item-item similarity matrix computed using collaborative filtering techniques.

8. **Get Best Hyperparameters:**
   - After fitting the data, we retrieve the best hyperparameters selected by the grid search using the `best_params_` attribute of the `grid_search` object.

9. **Print Best Parameters:**
   - Finally, we print the best hyperparameters obtained from the grid search.



`scikit-learn (sklearn)`:

- Scikit-learn is a popular machine learning library in Python that provides simple and efficient tools for data analysis and modeling.
- It includes various modules for tasks such as classification, regression, clustering, dimensionality reduction, and model selection.
- The GridSearchCV class from scikit-learn is used for hyperparameter tuning through grid search along with cross-validation.
- The make_scorer function allows us to create a custom scoring function for use with GridSearchCV.
- The NearestNeighbors class provides functionality for unsupervised nearest neighbors learning, which can be used for tasks such as finding k-nearest neighbors for a given data point.

In [43]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import make_scorer
import numpy as np

# Define a custom scorer based on cosine similarity defined above
cosine_similarity_scorer = make_scorer(cosine_similarity)

# Define the parameter grid
param_grid = {
    'n_neighbors': [5, 10, 15, 30, 40],
    'metric': ['cosine', 'euclidean']
}

# Initialize NearestNeighbors model
knn_model = NearestNeighbors()

# Create the GridSearchCV object
grid_search = GridSearchCV(knn_model, param_grid, cv=5, scoring=cosine_similarity_scorer)

# Fit the data to perform hyperparameter tuning
grid_search.fit(train_matrix)  # train_matrix contains item-user matrix for the train set

# Get the best hyperparameters
best_params = grid_search.best_params_

print("Best parameters:", best_params)


c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_

GridSearchCV(cv=5, estimator=NearestNeighbors(),
             param_grid={'metric': ['cosine', 'euclidean'],
                         'n_neighbors': [5, 10, 15, 30, 40]},
             scoring=make_scorer(cosine_similarity, response_method='predict'))

Best parameters: {'metric': 'cosine', 'n_neighbors': 5}


This code sets up a GridSearchCV object to perform hyperparameter tuning using the train set (train_matrix). It explores different combinations of hyperparameters specified in the param_grid, evaluates them using 5-fold cross-validation (cv=5), and uses the cosine_similarity scorer to optimize the model's performance based on cosine similarity. Finally, it prints the best hyperparameters found during the search.

The best parameter found is the one we trained the model with, so we don't have to train the model again.

We have to perform the calculation of the predicted ratings for the validation set to generate predicted ratings in order to evaluate the performance of our trained model on a validation dataset. 

If we computed the similarity matrix again specifically for the validation set, it would essentially mean that we are using a different set of similarity measures for predicting ratings compared to what we used during training. This approach could lead to inconsistencies and potentially degrade the performance of our model. Here's what could happen:

1. **Inconsistency**: The similarity measures computed for the train_val set might differ from those computed for the training set due to variations in the data. As a result, the predicted ratings based on these new similarity measures may not align well with the predictions made during training, leading to inconsistency in the model's behavior.

2. **Overfitting**: Computing a new similarity matrix specifically for the train_val set might lead to overfitting on the validation data. The model may capture noise or idiosyncrasies present in the train_val set, which may not generalize well to unseen data.

3. **Increased Complexity**: Computing the similarity matrix again for the train_val set adds computational complexity and redundancy, especially if the similarity computation process is resource-intensive. This can result in longer training times and increased resource utilization.

Overall, it's generally recommended to use the same similarity measures or neighborhood definitions for both training and validation sets to ensure consistency and generalizability of the model.

In [44]:
# Initialize an empty DataFrame to store predicted ratings for train_val
predicted_ratings_val = pd.DataFrame(index=val_matrix.index, columns=val_matrix.columns)

# Iterate over each user and their ratings in the train_val set
for user_id, user_ratings in val_matrix.iterrows():
    for movie_id, rating in user_ratings.items():
        
        # Check if the movie has a neighborhood defined
        if movie_id in item_neighborhoods_train:
            neighborhood = item_neighborhoods_train[movie_id]
            
            # Filter out movies from the neighborhood that the user has rated in the train set
            filtered_neighborhood = [neighbor_movie_id for neighbor_movie_id in neighborhood if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0]
            
            # Check if there are valid indices in the filtered neighborhood
            if len(filtered_neighborhood) > 0:
                # Calculate the predicted rating for the target movie based on the neighborhood
                neighbor_ratings = [user_ratings.loc[neighbor_movie_id] for neighbor_movie_id in filtered_neighborhood]
                
                # Calculate the mean rating using only the movies in the neighborhood that have been rated by the user
                predicted_rating = np.mean(neighbor_ratings)
            else:
                # If the filtered neighborhood is empty, assign the mean rating of all movies rated by the user
                predicted_rating = user_ratings[user_ratings != 0].mean()
            
            # Assign the predicted rating to the corresponding cell in the DataFrame
            predicted_ratings_val.at[user_id, movie_id] = predicted_rating

# Fill NaN values with mean ratings across all users
predicted_ratings_val.fillna(predicted_ratings_val.mean().mean(), inplace=True)


In [45]:
predicted_ratings_val.head()

predicted_ratings_val.shape

,75,77,135,208,290,413,461,733,792,810,...,271162,272579,274245,274495,277820,280482,283259,284167,284513,286031
18425,4.0,4.0,4.0,4.0,4.0,4.0,4.0,3.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
127354,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0
83656,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,...,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333
80513,4.625,4.625,4.625,4.5,4.625,4.625,4.625,4.5,4.625,4.625,...,4.625,4.5,4.625,4.625,4.625,4.625,4.625,4.625,4.625,4.625
16569,4.0,4.0,4.0,4.0,4.5,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.75,4.0,4.0


(379, 435)

Now we evaluate the model with the Validation item-user matrix with the recently calculated predicted ratings for the validation set.

In [46]:
# Now, we can evaluate the model using the evaluate_model function
mae, rmse = evaluate_model(val_matrix.to_numpy(), predicted_ratings_val)

print("Mean Absolute Error (MAE) on Validation Data:", mae)
print("Root Mean Squared Error (RMSE) on Validation Data:", rmse)


Mean Absolute Error (MAE) on Validation Data: 0.7000548313924697
Root Mean Squared Error (RMSE) on Validation Data: 0.9463766360957415


The results show that the model performs even better than the training set. This can be due to the fact the the overall opinion in the dataset is the same as the one in the training dataset. Meaning that the predicted ratings in the validation set are very similar to the true ones in the val set because there is like a popularity bias of the items users like the same items. 

### Retraining of the model with the Train-Val set

Once the have calculated and evaluated the model on the validation data, we have retrain the similarity matrix for the train-val data. This means that we have to recalculate the similarity between items now using the reviews in train and validation and also the genre similarity matrix between items which remains the same, since the items in `item_similarity_genres_matrix` don't change.

In [47]:
# After the model evaluation on the validation set, we recalculate the similarity matrix
# of the model on the train-val data
item_similarity_matrix_train_val = calculate_item_similarity_matrix(train_val)

# We combine the similarity matrices (the retrained one and the genres)
item_similarity_matrix_train_val = combine_similarity_matrices(item_similarity_matrix_train_val, item_similarity_genres_matrix)

# We calculate the item neighborhoods for the train_val data 
item_neighborhoods_train_val = neighborhood_selection(item_user_array, index_to_movie_id, item_similarity_matrix_train_val)

### Step 7: Deployment

Once we have come up with the best parameters possible and trained the model with the whole train_validation set, we will test it on the test set.

In [48]:
test_matrix

,75,77,135,208,290,413,461,733,792,810,...,271162,272579,274245,274495,277820,280482,283259,284167,284513,286031
194282,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
61655,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
166208,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188063,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
282226,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236712,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
57968,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
285199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315886,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
# Check if there are NaN values in the test matrix
test_matrix.isnull().sum().sum()

0

In [50]:
# Now, we can evaluate the model's predicted ratings on the test set

# Initialize an empty DataFrame to store predicted ratings
predicted_ratings_test = pd.DataFrame(index=test_matrix.index, columns=test_matrix.columns)

# Iterate over each user-item pair in the training set
for user_id, user_ratings in test_matrix.iterrows():
    for movie_id, rating in user_ratings.items():
    
        
        # Check if the movie has a neighborhood defined
        if movie_id in item_neighborhoods_train_val:
            neighborhood = item_neighborhoods_train_val[movie_id]
            
            # Filter out movies from the neighborhood that the user has rated in the training set
            filtered_neighborhood = [neighbor_movie_id for neighbor_movie_id in neighborhood if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0]
            
            # Check if there are valid indices in the filtered neighborhood
            if len(filtered_neighborhood) > 0:
                # Calculate the predicted rating for the target movie based on the neighborhood
                neighbor_ratings = [user_ratings.loc[neighbor_movie_id] for neighbor_movie_id in filtered_neighborhood]
                
                # Calculate the mean rating using only the movies in the neighborhood that have been rated by the user
                predicted_rating = np.mean(neighbor_ratings)
            else:
                # If the filtered neighborhood is empty, assign the mean rating of all movies rated by the user
                predicted_rating = user_ratings[user_ratings != 0].mean()
            
            # Assign the predicted rating to the corresponding cell in the DataFrame
            predicted_ratings_test.at[user_id, movie_id] = predicted_rating

# Fill NaN values with mean ratings across all users
predicted_ratings_test.fillna(predicted_ratings_test.mean().mean(), inplace=True)

# Now, we can evaluate the model's predicted ratings on the test set
mae, rmse = evaluate_model(test_matrix.to_numpy(), predicted_ratings_test.to_numpy())

print("Mean Absolute Error (MAE) on Test Data using the Ratings given to the Test Set:", mae)
print("Root Mean Squared Error (RMSE) on Test Data using the Ratings given to the Test Set:", rmse)


Mean Absolute Error (MAE) on Test Data using the Ratings given to the Test Set: 0.7041303026316839
Root Mean Squared Error (RMSE) on Test Data using the Ratings given to the Test Set: 0.9316186182877294


Overall the performance of the model seems very good, as it is performing very well on predicting data for unseen users showcasing a small MAE and RMSE.

In [51]:
top_recommendations_test = []

# Iterate over each row (user) in the test_matrix
for user_id in test_matrix.index:
    # Get the actual user ID corresponding to the user index
    recommendations = get_top_recommendations(user_id, predicted_ratings_test, df)
    top_recommendations_test.append(recommendations)

# Output: List of top movie recommendations for each user in the test matrix
top_recommendations_test

# print size of top_recommendations
print("Size of top_recommendations_test:", len(top_recommendations_test))



[["As If I Didn't Exist (Elina - Som om jag inte fanns) ",
  'Ten Minutes Older: The Trumpet ',
  'Finsterworld ',
  'Jeanne la Pucelle II - Les prisons ',
  'Jara Cimrman Lying, Sleeping '],
 ['Night Games ',
  'Wreck of the Mary Deare, The ',
  'Hustler, The ',
  'Soft Matter ',
  'The Roost '],
 ['WWE Royal Rumble 2022 ',
  'Deliverance ',
  'Eros ',
  'An Inspector Calls ',
  "Who'll Stop the Rain "],
 ['Transformers: Dark of the Moon ',
  'Waterworld ',
  'What Dreams May Come ',
  'Death Race ',
  'FM '],
 ['Transformers: Dark of the Moon ',
  'Waterworld ',
  'What Dreams May Come ',
  'Death Race ',
  'FM '],
 ['W. ',
  'Ajeeb Daastaans ',
  'American Woman ',
  'Hustler, The ',
  'Death of Mr. Lazarescu, The (Moartea domnului Lazarescu) '],
 ['Godfather: Part II, The ',
  'Silence ',
  'That Girl in Yellow Boots ',
  'Scent of a Woman ',
  'Zulu '],
 ['Night Games ',
  'Rock, The ',
  'Hustler, The ',
  'The Roost ',
  'Warriors, The '],
 ['Pursuit of Happyness, The ',
  'Slip

Size of top_recommendations_test: 474


# Item-based Classification (ItemKNN)

### Step 1: Data preparation

We have created already the user-item interactions matrix (`item_user_matrix`) in the first part of the Regression model above.

Therefore, this step is already done in this model.

For this step we are going to resuse the `item_user_matrix` that we created in the first model built. 

### Step 2: Compute Item Similarity

The similarity matrices both for the genres (`item_similarity_genres_matrix`) and for the ratings (`item_similarity_matrix_train` and `item_similarity_matrix_train_val`) were already created in Step 2 of the Regression model above.


### Step 3: Define items neighborhood

- Determine the neighborhood size, i.e., the number of most similar items to consider when predicting ratings for a target item.
- Select the most similar items for each item in the dataset based on their calculated similarities. This forms the neighborhood for each item.

#### Item-Based Neighborhoods and Ratings Aggregation

This code following snippet enhances the previous item-based collaborative filtering approach by considering ratings aggregation within the item neighborhoods.

##### Steps:

1. **Defining Neighborhood Size**:
   - The variable `k` determines the number of most similar items to consider in the neighborhood.

2. **Initializing Data Structure**:
   - An empty dictionary `item_neighborhoods_classification_train` is initialized to store the neighborhoods for each item.

3. **Iterating Over Items**:
   - For each movie in the dataset:
     - All ratings for the current movie are extracted from the DataFrame (`df`).
     - Ratings aggregation is performed. In this example, the average rating for the movie is computed, but other aggregation methods can be used.
     - The similarity scores for the current movie are retrieved from the precomputed `item_similarity_matrix_train`.
     - Similarity scores are sorted in descending order, and the indices of the most similar items (excluding itself) are obtained.
     - These indices are converted back to movie IDs, forming the neighborhood for the current item.
     - The neighborhood for the current item is stored in the `item_neighborhoods` dictionary.

4. **Output**:
   - `item_neighborhoods`: A dictionary where keys are movie IDs, and values are lists of movie IDs representing the neighborhood of each item. Each movie's neighborhood includes movies with similar ratings and content.

##### Note:
- This approach considers both similarity in ratings and content (as captured by cosine similarity) when building item neighborhoods.
- Aggregating ratings within item neighborhoods helps in providing more personalized recommendations.

In [52]:
import numpy as np

def item_neighborhood_selection(similarity_matrix, k=None, threshold=None, item_ids=None):
    """
    Select a subset of similar items for each item based on similarity matrix.

    Parameters:
        similarity_matrix (numpy.ndarray): Item-item similarity matrix.
        k (int): Number of similar items to select (optional).
        threshold (float): Similarity threshold for selecting similar items (optional).
        item_ids (list): List of item IDs corresponding to rows/columns of the similarity matrix.

    Returns:
        dict: Dictionary containing similar items for each item.
    """
    num_items = similarity_matrix.shape[0]
    item_neighborhood = {}

    for i in range(num_items):
        if k is not None:
            # Select top-k similar items (excluding the item itself)
            similar_items_indices = np.argsort(similarity_matrix[i])[::-1][:k]
        elif threshold is not None:
            # Select items with similarity above threshold (excluding the item itself)
            similar_items_indices = np.where(similarity_matrix[i] > threshold)[0]

        # Remove the item itself from the neighborhood
        similar_items_indices = similar_items_indices[similar_items_indices != i]

        # Get the item ID corresponding to the current index
        current_item_id = item_ids[i] if item_ids is not None else i

        # Get the item IDs for the neighborhood
        neighborhood_item_ids = [item_ids[index] for index in similar_items_indices]

        # Store the similar items with the item's ID as the key
        item_neighborhood[current_item_id] = neighborhood_item_ids

    return item_neighborhood

# We have an item-item similarity matrix 'item_similarity_matrix_train'
# and we want to select top-5 similar items for each item
k = 5
item_neighborhoods_classification_train = item_neighborhood_selection(item_similarity_matrix_train, k=k, item_ids=index_to_movie_id)

# Check if there are any items with empty neighborhoods
empty_neighborhoods = []

for item_id, neighborhood in item_neighborhoods_classification_train.items():
    if not neighborhood:
        empty_neighborhoods.append(item_id)

if empty_neighborhoods:
    print("Items with empty neighborhoods:", empty_neighborhoods)
else:
    print("No items with empty neighborhoods found.")


No items with empty neighborhoods found.


In [53]:
len(item_neighborhoods_classification_train)

435

### Step 4: Ratings classification selection

This is the step that differs the most from the regression model. 

In the code below, repeats the process from the rating prediction model, but in this case, we select the most similar movies for each item and we try selecting the most similar rating available, this is where the classification part comes from. We will classify the items per neighborhood and select the most similar item in that neighborhood that has an available rating.

In [54]:
import numpy as np
import pandas as pd

def recommend_movies(train_matrix, item_neighborhoods, movie_id_to_index, num_movies=5):
    """
    Recommend top movies for each user based on item neighborhood.

    Parameters:
        train_matrix (np.ndarray or pd.DataFrame): Matrix containing user-item ratings in the train set.
        item_neighborhoods (dict): Dictionary containing similar items for each item.
        movie_id_to_index (dict): Dictionary mapping movie IDs to their corresponding indices in the train_matrix.
        num_movies (int): Number of movies to recommend for each user.

    Returns:
        pd.DataFrame: DataFrame containing predicted ratings for recommended movies for each user.
    """
    # Convert train_matrix to DataFrame if it's a numpy array
    if isinstance(train_matrix, np.ndarray):
        train_matrix = pd.DataFrame(train_matrix, index=np.arange(train_matrix.shape[0]), columns=np.arange(train_matrix.shape[1]))

    # Initialize an empty DataFrame to store predicted ratings
    predicted_ratings = pd.DataFrame(index=train_matrix.index, columns=train_matrix.columns)

    # Iterate over each user-item pair in the training set
    for user_id, user_ratings in train_matrix.iterrows():
        for movie_id, rating in user_ratings.items():
            
            # Skip if the rating is non-zero (indicating a rating given by the user)
            if rating != 0:
                continue
            
            # Check if the movie has a neighborhood defined
            if movie_id in item_neighborhoods:
                neighborhood = item_neighborhoods[movie_id]
                
                # Initialize a flag to track if the rating has been predicted
                rating_predicted = False
                
                # Iterate over the items in the neighborhood
                for neighbor_movie_id in neighborhood:
                    # Check if the neighbor movie has been rated by the user
                    if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0:
                        # Use the rating of the closest item in the neighborhood
                        predicted_rating = user_ratings.loc[neighbor_movie_id]
                        rating_predicted = True
                        break
                
                # If none of the items in the neighborhood have been rated, use the user's average rating
                if not rating_predicted:
                    if user_ratings[user_ratings != 0].empty:
                        # If the user hasn't rated any movies, use the global mean rating
                        predicted_rating = train_matrix[train_matrix != 0].mean().mean()
                    else:
                        # Use the average of the ratings given by the user
                        predicted_rating = user_ratings[user_ratings != 0].mean()
                
                # Assign the predicted rating to the corresponding cell in the DataFrame
                predicted_ratings.at[user_id, movie_id] = predicted_rating

    # Fill NaN values with mean ratings across all users
    predicted_ratings.fillna(predicted_ratings.mean().mean(), inplace=True)

    # Set the index of the DataFrame as the user ID of the train set
    predicted_ratings.index = train_matrix.index

    # Display the head of the predicted_ratings DataFrame
    print(predicted_ratings.head())

    return predicted_ratings


# Recommend movies for each user in the train set
predicted_ratings_classification_train = recommend_movies(train_matrix, item_neighborhoods_classification_train, movie_id_to_index)

# Print the shape of the predicted ratings matrix
print("Shape of predicted ratings matrix:", predicted_ratings_classification_train.shape)


          75        77        135       208       290       413       461     \
120864  3.000000  3.000000  3.000000  3.360871  3.000000  3.000000  3.000000   
91440   3.500000  3.500000  3.500000  3.500000  3.500000  3.500000  3.500000   
58822   3.100000  3.100000  3.100000  3.000000  3.500000  3.100000  3.100000   
19119   3.333333  3.333333  3.333333  3.333333  3.333333  3.333333  3.333333   
174808  2.500000  2.750000  3.500000  3.360871  2.750000  3.360871  2.750000   

          733       792       810     ...    271162    272579    274245  \
120864  3.000000  3.000000  3.000000  ...  3.000000  3.000000  3.000000   
91440   3.500000  3.500000  3.500000  ...  3.500000  3.500000  3.500000   
58822   4.000000  3.100000  3.100000  ...  2.000000  4.500000  3.100000   
19119   3.333333  3.333333  3.333333  ...  3.333333  3.333333  3.333333   
174808  2.750000  2.750000  2.750000  ...  2.750000  2.750000  2.750000   

        274495    277820    280482    283259    284167    284513    

C:\Users\Jaume\AppData\Local\Temp\ipykernel_10804\1007744238.py:61: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  predicted_ratings.fillna(predicted_ratings.mean().mean(), inplace=True)


In [55]:
type(train_matrix)

pandas.core.frame.DataFrame

In [56]:
predicted_ratings_classification_train

,75,77,135,208,290,413,461,733,792,810,...,271162,272579,274245,274495,277820,280482,283259,284167,284513,286031
120864,3.000000,3.000000,3.000000,3.360871,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
91440,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,...,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000
58822,3.100000,3.100000,3.100000,3.000000,3.500000,3.100000,3.100000,4.000000,3.100000,3.100000,...,2.000000,4.500000,3.100000,3.100000,3.100000,3.100000,3.100000,3.500000,3.100000,3.100000
19119,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,...,3.333333,3.333333,3.333333,3.000000,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333
174808,2.500000,2.750000,3.500000,3.360871,2.750000,3.360871,2.750000,2.750000,2.750000,2.750000,...,2.750000,2.750000,2.750000,2.750000,2.750000,2.750000,2.750000,2.750000,2.750000,2.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96001,2.000000,2.500000,2.500000,4.000000,2.500000,2.500000,2.500000,3.360871,2.500000,2.500000,...,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,4.000000
210321,2.800000,2.800000,2.800000,2.800000,2.800000,2.800000,2.800000,2.800000,2.800000,2.800000,...,2.800000,2.800000,3.360871,2.800000,2.800000,2.800000,2.800000,2.800000,2.800000,2.800000
19965,3.666667,3.666667,3.666667,3.000000,3.666667,3.666667,3.666667,3.360871,3.666667,3.666667,...,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.666667,3.000000
222135,2.333333,2.333333,2.333333,3.360871,2.333333,2.333333,2.333333,3.360871,2.333333,2.333333,...,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333,2.333333,2.000000


### Recommendations Generation

In [57]:
import pandas as pd

# Convert predicted_ratings_classification_train to a pandas DataFrame if it's a numpy array
if isinstance(predicted_ratings_classification_train, np.ndarray):
    # Use the length of train_matrix as the number of rows and items as columns
    num_users, num_items = train_matrix.shape
    predicted_ratings_classification_train = pd.DataFrame(predicted_ratings_classification_train, index=range(num_users), columns=range(num_items))

# Initialize a dictionary to store top recommendations for each user
top_recommendations_per_user = {}

# Iterate over each user in the train matrix
for user_id in train_matrix.index:
    # Get the top recommendations for the current user using predicted_ratings_classification_train
    recommendations = get_top_recommendations(user_id, predicted_ratings_classification_train, df)
    
    # Store the recommendations in the dictionary
    top_recommendations_per_user[user_id] = recommendations

# Print the top 5 recommendations for each user
for user_id, recommendations in top_recommendations_per_user.items():
    print(f"User {user_id} recommendations:", recommendations)


User 120864 recommendations: ['Transformers: Dark of the Moon ', 'Horton Hears a Who! ', 'Office ', 'Uncle P ', 'Lorenzo ']
User 91440 recommendations: ['The King and I ', 'Happy Feet ', 'Nine Days ', 'Coquette ', 'Hillbilly Elegy ']
User 58822 recommendations: ['The Cleveland Kidnappings ', 'The Jade Mask ', 'Magical Disappearing Money ', 'Adrenaline ', "President's Last Bang, The (Geuddae geusaramdeul) "]
User 19119 recommendations: ['Big Bully ', 'Joe Versus the Volcano ', "New Year's Eve ", 'Unconscious (Inconscientes) ', 'Woo ']
User 174808 recommendations: ['Village People ', "Who's Your Caddy? ", 'Balls Out ', 'Siberia ', 'Down Periscope ']
User 120265 recommendations: ['Lokis. A Manuscript of Professor Wittembach ', 'Tehran: City of Love ', 'Sleepy Hollow ', 'Matchmaking ', 'Christmas at Dollywood ']
User 314087 recommendations: ['Godzilla vs. Gigan (Chikyû kogeki meirei: Gojira tai Gaigan) ', 'Thief of Bagdad, The ', 'Iron Man & Captain America: Heroes United ', 'Heller in Pin

### Step 5: Model Evaluation (Item KNN Classification)

We are going to evaluate the model here using the Mean Absolute Error (MAE) and Real Mean Squared Error (RMSE), comparing the predicted rartings vs the true ratings given by the users.

In [58]:
import numpy as np
import pandas as pd

# Ensure predicted_ratings_classification_train is a numpy array
if isinstance(predicted_ratings_classification_train, pd.DataFrame):
    predicted_ratings_classification_train = predicted_ratings_classification_train.to_numpy()

# Assuming train_matrix is the numpy array representing the item-user matrix

# Ensure train_matrix is a 2D array
if isinstance(train_matrix, pd.DataFrame):
    train_matrix = train_matrix.to_numpy()

if train_matrix.ndim == 1:
    train_matrix = np.expand_dims(train_matrix, axis=0)

# Evaluate the model
mae, rmse = evaluate_model(train, predicted_ratings_classification_train)

print("Mean Absolute Error (MAE) on Training Data:", mae)
print("Root Mean Squared Error (RMSE) on Training Data:", rmse)


Mean Absolute Error (MAE) on Training Data: 0.8524951428587664
Root Mean Squared Error (RMSE) on Training Data: 1.070593319383507


### Step 6: Parameter Tuning

1. **Import Necessary Libraries:**
   - We import the required libraries for performing grid search cross-validation (`GridSearchCV`), creating custom scorers (`make_scorer`), and utilizing the `NearestNeighbors` algorithm.

2. **Define Cosine Similarity Function:**
   - We define a custom function `cosine_similarity` to compute the cosine similarity between two vectors. This function calculates the dot product of the vectors and divides it by the product of their norms.

3. **Define Custom Scorer:**
   - We create a custom scorer `cosine_similarity_scorer` using `make_scorer`, which enables us to use cosine similarity as the scoring metric during grid search cross-validation.

4. **Define Parameter Grid:**
   - We specify a parameter grid `param_grid` containing the hyperparameters to be tuned. In this case, we're tuning the number of neighbors (`n_neighbors`) and the distance metric (`metric`) for the `NearestNeighbors` algorithm.

5. **Initialize NearestNeighbors Model:**
   - We initialize the `NearestNeighbors` model without specifying any hyperparameters.

6. **Create GridSearchCV Object:**
   - We create a `GridSearchCV` object named `grid_search` with the specified parameter grid, cross-validation strategy (5-fold cross-validation), and custom scoring metric (`cosine_similarity_scorer`).

7. **Fit the Data:**
   - We fit the `item_user_matrix` data to the `grid_search` object to perform hyperparameter tuning. `item_user_matrix` typically contains the item-item similarity matrix computed using collaborative filtering techniques.

8. **Get Best Hyperparameters:**
   - After fitting the data, we retrieve the best hyperparameters selected by the grid search using the `best_params_` attribute of the `grid_search` object.

9. **Print Best Parameters:**
   - Finally, we print the best hyperparameters obtained from the grid search.



In [59]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import make_scorer
import numpy as np

# Define a custom scorer based on cosine similarity defined above
cosine_similarity_scorer = make_scorer(cosine_similarity)

# Define the parameter grid
param_grid = {
    'n_neighbors': [5, 10, 15, 30, 40],
    'metric': ['cosine', 'euclidean']
}

# Initialize NearestNeighbors model
knn_model = NearestNeighbors()

# Create the GridSearchCV object
grid_search = GridSearchCV(knn_model, param_grid, cv=5, scoring=cosine_similarity_scorer)

# Fit the data to perform hyperparameter tuning
grid_search.fit(train_matrix)  # val_matrix contains item-user matrix

# Get the best hyperparameters
best_params = grid_search.best_params_

print("Best parameters:", best_params)


c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Jaume\anaconda3\envs\SDM\Lib\site-packages\sklearn\model_selection\_validation.py", line 980, in _score
    scores = scorer(estimator, X_test, **score_

GridSearchCV(cv=5, estimator=NearestNeighbors(),
             param_grid={'metric': ['cosine', 'euclidean'],
                         'n_neighbors': [5, 10, 15, 30, 40]},
             scoring=make_scorer(cosine_similarity, response_method='predict'))

Best parameters: {'metric': 'cosine', 'n_neighbors': 5}


The hyperparameter tuning gives the same results as for the Ratings prediction model as expected. We are going to now evaluate the model with the best found parameters (which we have already trained to model on) on the validation set.

We are now going to obtain again the ratings for the full val set to evaluate the performance. 

In [60]:
# Function to recalculate predictions for a given matrix using the trained model
def recalculate_predictions(matrix, item_neighborhoods):
    """
    Recalculate predictions for a given matrix using the trained model.

    Parameters:
        matrix (pd.DataFrame): Matrix containing user-item ratings.
        item_neighborhoods: A dictionary containing item neighborhoods for each item in the dataset.

    Returns:
        pd.DataFrame: DataFrame containing recalculated predicted ratings for each user.
    """
    # Initialize an empty DataFrame to store recalculated predicted ratings
    recalculated_predictions = pd.DataFrame(index=matrix.index, columns=matrix.columns)

    # Iterate over each user-item pair in the matrix
    for user_id, user_ratings in matrix.iterrows():
        for movie_id, rating in user_ratings.items():
            # Skip if the rating is non-zero (indicating a rating given by the user)
            if rating != 0:
                continue

            # Check if the movie has a neighborhood defined
            if movie_id in item_neighborhoods:
                neighborhood = item_neighborhoods[movie_id]

                # Initialize a flag to track if the rating has been predicted
                rating_predicted = False

                # Iterate over the items in the neighborhood
                for neighbor_movie_id in neighborhood:
                    # Check if the neighbor movie has been rated by the user
                    if neighbor_movie_id in user_ratings.index and user_ratings.loc[neighbor_movie_id] != 0:
                        # Use the rating of the closest item in the neighborhood
                        predicted_rating = user_ratings.loc[neighbor_movie_id]
                        rating_predicted = True
                        break

                # If none of the items in the neighborhood have been rated, use the user's average rating
                if not rating_predicted:
                    if user_ratings[user_ratings != 0].empty:
                        # If the user hasn't rated any movies, use the global mean rating
                        predicted_rating = matrix[matrix != 0].mean().mean()
                    else:
                        # Use the average of the ratings given by the user
                        predicted_rating = user_ratings[user_ratings != 0].mean()

                # Assign the predicted rating to the corresponding cell in the DataFrame
                recalculated_predictions.at[user_id, movie_id] = predicted_rating

    # Fill NaN values with mean ratings across all users
    recalculated_predictions.fillna(recalculated_predictions.mean().mean(), inplace=True)

    return recalculated_predictions


# Recalculate predictions for the validation set (train_val_matrix)
recalculated_val_predictions = recalculate_predictions(val_matrix, item_neighborhoods_classification_train)


C:\Users\Jaume\AppData\Local\Temp\ipykernel_10804\712862863.py:52: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  recalculated_predictions.fillna(recalculated_predictions.mean().mean(), inplace=True)


In [61]:
recalculated_val_predictions

,75,77,135,208,290,413,461,733,792,810,...,271162,272579,274245,274495,277820,280482,283259,284167,284513,286031
18425,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,3.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
127354,4.000000,3.379362,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000
83656,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,...,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333
80513,4.625000,4.625000,4.625000,4.500000,4.625000,4.625000,4.625000,4.625000,4.625000,4.625000,...,4.625000,4.500000,4.625000,4.625000,4.625000,4.625000,4.625000,4.625000,4.625000,4.625000
16569,4.000000,4.000000,4.000000,3.379362,5.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220310,3.375000,3.375000,3.375000,3.000000,4.000000,3.375000,3.375000,3.375000,3.375000,3.375000,...,3.375000,3.375000,3.375000,3.375000,3.375000,3.375000,3.375000,4.000000,3.375000,3.375000
271337,3.166667,3.166667,3.166667,3.166667,3.000000,3.166667,3.166667,3.166667,3.166667,3.166667,...,3.166667,3.166667,3.166667,3.166667,3.166667,3.166667,3.166667,3.000000,3.166667,3.166667
46623,2.166667,2.166667,2.166667,2.166667,5.000000,2.166667,2.166667,2.166667,2.166667,2.166667,...,2.166667,2.166667,2.166667,2.166667,2.166667,2.166667,2.166667,2.166667,2.166667,2.166667
306413,3.375000,3.375000,3.375000,3.375000,4.000000,3.375000,3.375000,3.375000,3.375000,3.375000,...,3.375000,3.375000,3.375000,3.375000,3.375000,3.375000,3.375000,4.000000,3.375000,3.375000


In [62]:
# Evaluate the model
mae, rmse = evaluate_model(val, recalculated_val_predictions)

print("Mean Absolute Error (MAE) on Validation Data:", mae)
print("Root Mean Squared Error (RMSE) on Validation Data:", rmse)

Mean Absolute Error (MAE) on Validation Data: 0.844680389869479
Root Mean Squared Error (RMSE) on Validation Data: 1.0691360931628504


### Step 7: Deployment

Once we have all the metrics, we have to recalculate the ratings for the test set using the calculated `item_similarity_matrix_train_val`, which we created in the Ratings prediction model. After getting the latest item neighborhoods we are going to repeat the same process (get the closest rating available) for the test set and evaluate the model.

In [63]:
# We have to reobtain the neighborhood for the train_val data
# We will use the already calculated item-item similarity matrix for train_val data
item_neighborhoods_classification_train_val = item_neighborhood_selection(item_similarity_matrix_train_val, k=5, item_ids=index_to_movie_id)

# Recalculate predictions for the validation set (test matrix)
recalculated_test_predictions = recalculate_predictions(test_matrix, item_neighborhoods_classification_train_val)

C:\Users\Jaume\AppData\Local\Temp\ipykernel_10804\712862863.py:52: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  recalculated_predictions.fillna(recalculated_predictions.mean().mean(), inplace=True)


We also have to evaluate the metrics for the test set.

In [64]:
# Evaluate the model
mae, rmse = evaluate_model(test, recalculated_test_predictions)

print("Mean Absolute Error (MAE) on Test Data:", mae)
print("Root Mean Squared Error (RMSE) on Test Data:", rmse)

Mean Absolute Error (MAE) on Test Data: 0.850547055706049
Root Mean Squared Error (RMSE) on Test Data: 1.0748622911313526


## Conclusions

After thoroughly conducting the process of training, validating, and testing both models, it is evident that the regression model outperforms the alternative. This conclusion is drawn based on its ability to utilize the averages of available ratings effectively, enabling it to generate accurate predictions for movie recommendations. 

Through comprehensive evaluation, the regression model consistently demonstrated superior performance metrics compared to the Classification model considered in this study. Its prediction works better than the selection of the clossest neighbor in the classification model and it offers a robust mechanism for predicting user preferences and recommending relevant movies. Therefore, in the context of this study, the regression model emerges as the preferred choice for movie recommendation systems.

After running both the models with genres and without we can say that the one with the genres performs better in the evaluation metrics therefore offers more accurate prediction to the real ones than the model without genres, this is interesting because it can mean that the content add more reliablity than the actual ratings given by the users (even though we are still giving the ratings a higher value in the sumamation of the similarities matrices).